In [ ]:
def load_data(file):
    with open (file, "r", encoding="utf-8") as f:
        data = json.load(f)
    return data

def write_data(file, data):
    with open (file, "w", encoding="utf-8") as f:
        json.dump(data, f, indent=4)

In [ ]:
def lemmatization(texts, allowed_postags=["NOUN", "ADJ", "VERB", "AV"]):
    nlp = spacy.load("en_core_web_sm", disable=["parser", "ner"])
    text_out = []
    for text in texts:
        doc = nlp(text)
        new_text = []
        for token in doc:
            if token.pos_ in allowed_postags:
                new_text.append(token.lemma_)
        final = " ".join(new_text)
        text_out.append(final)
    return text_out

In [ ]:
def preprocess_article(input_text):
    sentences_ted = []
    
    # Use regular expression to split the text into words
    sentences_ted = re.findall(r'\b\w+\b', input_text)
    sentences_ted = [token for token in sentences_ted if not token.isdigit()]


    # Load stopwords from a file into a set
    stoplist = set()
    with open('stopwords') as openfileobject: 
        for line in openfileobject:
            stoplist.add(line.strip())  # Use strip() to remove leading/trailing whitespace
    
    cleaned_text = " ".join(word for word in sentences_ted if word not in stoplist)

    return cleaned_text

In [ ]:
def gen_words(texts):
    final = [gensim.utils.simple_preprocess(text, deacc=True) for text in texts]
    return final

In [ ]:
def compute_coherence_values(dictionary, corpus, tokenizedData, limit, start=2, step=3):
    coherence_values = []
    model_list = []
    for num_topics in range(start, limit, step):
        model = gensim.models.ldamodel.LdaModel(corpus, num_topics=num_topics, id2word=dictionary, passes=10)
        #model = gensim.models.ldamulticore.LdaMulticore(corpus, num_topics = num_topics, id2word=dictionary,random_state=100,passes=10)
        model_list.append(model)
        coherencemodel = CoherenceModel(model=model, dictionary=dictionary, texts=tokenizedData, coherence='c_v')
        coherence_values.append(coherencemodel.get_coherence())

    return model_list, coherence_values